In [1]:
import awswrangler as wr

import mlflow

# Para que funciones, todos nuestros scripts debemos exportar las siguientes variables de entorno
%env AWS_ACCESS_KEY_ID=minio   
%env AWS_SECRET_ACCESS_KEY=minio123 
%env MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
%env AWS_ENDPOINT_URL_S3=http://localhost:9000

env: AWS_ACCESS_KEY_ID=minio
env: AWS_SECRET_ACCESS_KEY=minio123
env: MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
env: AWS_ENDPOINT_URL_S3=http://localhost:9000


In [2]:
mlflow_server = "http://localhost:5001"

mlflow.set_tracking_uri(mlflow_server)

In [3]:
import datetime

from mlflow.models import infer_signature
from neuralprophet import NeuralProphet
import optuna
from sklearn.metrics import mean_absolute_error

from mlflow_aux import get_or_create_experiment
from neural_prophet_wrapper import NeuralProphetWrapper
from optuna_aux import champion_callback, objective

# Optuna es un poco verboso, dejamos que solo nos muestre logs de errores
optuna.logging.set_verbosity(optuna.logging.ERROR)

/Users/alejandronicolastacchella/Documents/Personal/Maestría IA/MLOps1/ceia_mlops_1_tps/.venv/lib/python3.12/site-packages/lightning_fabric/__init__.py:29: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)
/Users/alejandronicolastacchella/Documents/Personal/Maestría IA/MLOps1/ceia_mlops_1_tps/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
/Users/alejandronicolastacchella/Documents/Personal/Maestría IA/MLOps1/ceia_mlops_1_tps/.venv/lib

In [4]:
raw_data_path = "s3://data/raw/trx_recaudacion_km_empresa_54_2020_2025.xlsx"
sheets = wr.s3.read_excel(raw_data_path, sheet_name=None)
lineas = sheets.keys()
# Número de pruebas por recorrido
N_TRIALS = 30
regresores = ["km"]

for linea in lineas:

    # Leemos datos de entrenamiento y validación
    df_train =  wr.s3.read_csv(f"s3://data/processed/{linea}/train.csv", parse_dates=["ds"])
    df_test =  wr.s3.read_csv(f"s3://data/processed/{linea}/test.csv", parse_dates=["ds"])

    experiment_id = get_or_create_experiment(f"Línea {linea}")
    print(experiment_id)

    run_name_parent = "best_hyperparam_"  + datetime.datetime.today().strftime('%Y/%m/%d-%H:%M:%S"')
    with mlflow.start_run(experiment_id=experiment_id, run_name=run_name_parent, nested=True):
        # Inicializamos el estudio de Optuna
        study = optuna.create_study(direction="minimize")

        # Ejecutamos los trials de optimización de hiperparametros. Cada uno de estos trials se ejecuta con un run separado, pero 
        # está anidado al run padre.
        study.optimize(
            lambda trial: objective(trial, df_train.copy(), df_test.copy(), regresores=regresores),
            n_trials=N_TRIALS, callbacks=[champion_callback]
        )

        # Una vez que terminamos la búsqueda, guardamos los mejores parámetros en el run padre.
        mlflow.log_params(study.best_params)
        mlflow.log_metric("best_train_mae", study.best_value)

        mlflow.set_tags(
            tags={
                "project": f"Línea {linea}",
                "optimizer_engine": "optuna",
                "model_family": "neuralprophet",
                "feature_set_version": 1,
            }
        )

        model = NeuralProphet(
            n_changepoints=study.best_params["n_changepoints"],
            trend_reg=study.best_params["trend_reg"],
            learning_rate=study.best_params["learning_rate"],
            seasonality_mode=study.best_params["seasonality_mode"],
            epochs=100,
            batch_size=128,
        )

        # Agregar regresores externos
        for reg in regresores:
            model.add_future_regressor(reg)

        # Entrenar modelo
        model.fit(df_train, freq='D')

        # Predecir sobre validación
        forecast = model.predict(df_test)

        mae = mean_absolute_error(df_test['y'].values, forecast['yhat1'].values)

        mlflow.log_metric("test_mae", mae)

        # Guardamos el artefacto del modelo
        artifact_path = f"linea_{linea}_model"

        # Firma de entrada/salida
        sample_input = df_train[["ds", "y"] + regresores].copy().tail(30)  # últimos 30 días
        sample_output = model.predict(sample_input)[["ds", "yhat1"]]
        signature = infer_signature(sample_input, sample_output)

        mlflow.pyfunc.log_model(
            python_model=NeuralProphetWrapper(model),
            name=artifact_path,
            signature=signature,
            registered_model_name=f"linea_{linea}_model_dev",
            metadata={"model_data_version": 1}
        )

        # Obtenemos la ubicación del modelo guardado en MLFlow
        model_uri = mlflow.get_artifact_uri(artifact_path)

    mlflow.end_run()

INFO - (NP.config.__post_init__) - Note: Trend changepoint regularization is experimental.
WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
/Users/alejandronicolastacchella/Documents/Personal/Maestría IA/MLOps1/ceia_mlops_1_tps/.venv/lib/python3.12/site-packages/neuralprophet/df_utils.py:464: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  if df.groupby("ID").apply(lambda x: x.duplicated("ds").any()).any():
/Users/alejandronicolastacchella/Documents/Personal/Maestría IA/MLOps1/ceia_mlops_1_tps/.venv/lib/python3.12/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a futur

1


/Users/alejandronicolastacchella/Documents/Personal/Maestría IA/MLOps1/ceia_mlops_1_tps/.venv/lib/python3.12/site-packages/pytorch_lightning/trainer/setup.py:201: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(


Epoch 100: 100%|██████████| 100/100 [00:00<00:00, 4966.73it/s, loss=0.0119, v_num=367, MAE=2.5e+3, RMSE=3.83e+3, Loss=0.012, RegLoss=0.000]    


/Users/alejandronicolastacchella/Documents/Personal/Maestría IA/MLOps1/ceia_mlops_1_tps/.venv/lib/python3.12/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.603% of the data.
/Users/alejandronicolastacchella/Documents/Personal/Maestría IA/MLOps1/ceia_mlops_1_tps/.venv/lib/python3.12/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)
/Users/alejandronicolastacchella/Documents/Personal/Maestría IA/MLOps1/ceia_mlops_1_tps/.venv/lib/python3.12/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.vi

...

/Users/alejandronicolastacchella/Documents/Personal/Maestría IA/MLOps1/ceia_mlops_1_tps/.venv/lib/python3.12/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 96.667% of the data.
/Users/alejandronicolastacchella/Documents/Personal/Maestría IA/MLOps1/ceia_mlops_1_tps/.venv/lib/python3.12/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)
/Users/alejandronicolastacchella/Documents/Personal/Maestría IA/MLOps1/ceia_mlops_1_tps/.venv/lib/python3.12/site-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.vi

Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 619.91it/s]


INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
/Users/alejandronicolastacchella/Documents/Personal/Maestría IA/MLOps1/ceia_mlops_1_tps/.venv/lib/python3.12/site-packages/mlflow/pyfunc/__init__.py:3262: UserWarning: An input example was not provided when logging the model. To ensure the model signature functions correctly, specify the `input_example` parameter. See https://mlflow.org/docs/latest/model/signatures.html#model-input-example for more details about the benefits of using input_example.
  color_warning(
Successfully registered model 'linea_504I_model_dev'.
2025/08/05 00:04:56 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: linea_504I_model_dev, version 1


🏃 View run best_hyperparam_2025/08/05-00:03:53" at: http://localhost:5001/#/experiments/9/runs/982499b1fd014b5f9770277c87d5af72
🧪 View experiment at: http://localhost:5001/#/experiments/9


Created version '1' of model 'linea_504I_model_dev'.
